In [1]:
import pandas as pd
import numpy as np


In [2]:
df= pd.read_excel("C:\\Users\\hp\\Downloads\\marketing_campaign_dataset.xlsx")

In [3]:
df.head()

,Campaign_ID,Company,Campaign_Type,Target_Audience,Duration,Channel_Used,Conversion_Rate,Acquisition_Cost,ROI,Location,Date,Clicks,Impressions,Engagement_Score,Customer_Segment
0,1,Innovate Industries,Email,Men 18-24,30 days,Google Ads,0.04,16174,6.29,Chicago,2021-01-01 00:00:00,506,1922,6,Health & Wellness
1,2,NexGen Systems,Email,Women 35-44,60 days,Google Ads,0.12,11566,5.61,New York,2021-02-01 00:00:00,116,7523,7,Fashionistas
2,3,Alpha Innovations,Influencer,Men 25-34,30 days,YouTube,0.07,10200,7.18,Los Angeles,2021-03-01 00:00:00,584,7698,1,Outdoor Adventurers
3,4,DataTech Solutions,Display,All Ages,60 days,YouTube,0.11,12724,5.55,Miami,2021-04-01 00:00:00,217,1820,7,Health & Wellness
4,5,NexGen Systems,Email,Men 25-34,15 days,YouTube,0.05,16452,6.50,Los Angeles,2021-05-01 00:00:00,379,4201,3,Health & Wellness


In [4]:
df.shape

(200005, 15)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200005 entries, 0 to 200004
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Campaign_ID       200005 non-null  int64  
 1   Company           200005 non-null  object 
 2   Campaign_Type     200005 non-null  object 
 3   Target_Audience   200005 non-null  object 
 4   Duration          200005 non-null  object 
 5   Channel_Used      200005 non-null  object 
 6   Conversion_Rate   200005 non-null  float64
 7   Acquisition_Cost  200005 non-null  int64  
 8   ROI               200005 non-null  float64
 9   Location          200005 non-null  object 
 10  Date              200005 non-null  object 
 11  Clicks            200005 non-null  int64  
 12  Impressions       200005 non-null  int64  
 13  Engagement_Score  200005 non-null  int64  
 14  Customer_Segment  200005 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 22.9+ MB


In [6]:
df.describe()

,Campaign_ID,Conversion_Rate,Acquisition_Cost,ROI,Clicks,Impressions,Engagement_Score
count,200005.000000,200005.000000,200005.000000,200005.000000,200005.000000,200005.000000,200005.000000
mean,100003.000000,0.080069,12504.441794,5.002416,549.774591,5507.307107,5.494673
std,57736.614632,0.040602,4337.663210,1.734485,260.019354,2596.863794,2.872593
min,1.000000,0.010000,5000.000000,2.000000,100.000000,1000.000000,1.000000
25%,50002.000000,0.050000,8740.000000,3.500000,325.000000,3266.000000,3.000000
50%,100003.000000,0.080000,12497.000000,5.010000,550.000000,5518.000000,5.000000
75%,150004.000000,0.120000,16264.000000,6.510000,775.000000,7753.000000,8.000000
max,200005.000000,0.150000,20000.000000,8.000000,1000.000000,10000.000000,10.000000


In [7]:
df = df.dropna()

In [8]:
df = df.drop_duplicates()


In [12]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [13]:
if 'conversions' not in df.columns:
    df['conversions'] = (df['conversion_rate'] * df['clicks']).round().astype(int)


In [14]:
df = df[
    (df['impressions'] >= df['clicks']) &
    (df['clicks'] >= df['conversions']) &
    (df['impressions'] >= 0) &
    (df['clicks'] >= 0) &
    (df['conversions'] >= 0) &
    (df['acquisition_cost'] >= 0)
]


In [15]:
df['ctr'] = np.where(
    df['impressions'] > 0,
    df['clicks'] / df['impressions'],
    0
)


In [16]:
df['cpc'] = np.where(
    df['clicks'] > 0,
    df['acquisition_cost'] / df['clicks'],
    0
)


In [17]:
campaign_performance = (
    df.groupby('campaign_id')
      .agg({
          'roi': 'mean',
          'impressions': 'sum',
          'clicks': 'sum',
          'conversions': 'sum',
          'acquisition_cost': 'sum'
      })
      .reset_index()
)


In [23]:
campaign_performance

,campaign_id,roi,impressions,clicks,conversions,acquisition_cost,performance
0,1,6.29,1922,506,20,16174,Good
1,2,5.61,7523,116,14,11566,Good
2,3,7.18,7698,584,41,10200,Good
3,4,5.55,1820,217,24,12724,Good
4,5,6.50,4201,379,19,16452,Good
...,...,...,...,...,...,...,...
200000,200001,2.84,5988,858,51,18365,Poor
200001,200002,4.14,3068,228,5,8168,Poor
200002,200003,3.25,9548,723,36,13397,Poor
200003,200004,3.86,2763,528,53,18508,Poor


In [18]:
avg_campaign_roi = campaign_performance['roi'].mean()

campaign_performance['performance'] = np.where(
    campaign_performance['roi'] >= avg_campaign_roi,
    'Good',
    'Poor'
)


In [19]:
high_spend_threshold = campaign_performance['acquisition_cost'].median()

high_spend_poor_campaigns = campaign_performance[
    (campaign_performance['acquisition_cost'] > high_spend_threshold) &
    (campaign_performance['performance'] == 'Poor')
]


In [20]:
campaign_funnel = (
    df.groupby('campaign_id')[['impressions', 'clicks', 'conversions']]
      .sum()
      .reset_index()
)

campaign_funnel['imp_to_click_drop'] = (
    1 - campaign_funnel['clicks'] / campaign_funnel['impressions']
)

campaign_funnel['click_to_conv_drop'] = (
    1 - campaign_funnel['conversions'] / campaign_funnel['clicks']
)


In [21]:
channel_performance = (
    df.groupby('channel_used')
      .agg({
          'roi': 'mean',
          'impressions': 'sum',
          'clicks': 'sum',
          'conversions': 'sum',
          'acquisition_cost': 'sum'
      })
      .reset_index()
      .sort_values(by='roi', ascending=False)
)


In [22]:
channel_performance

,channel_used,roi,impressions,clicks,conversions,acquisition_cost
1,Facebook,5.018672,180662496,18038175,1446343,410603426
4,Website,5.014114,183815901,18415351,1477696,416606897
2,Google Ads,5.003126,185020154,18342589,1468777,418944514
0,Email,4.996487,184801107,18493963,1485425,420874104
5,YouTube,4.993720,183450845,18350935,1463702,416797090
3,Instagram,4.988706,183738455,18316654,1462938,417124850


In [24]:
campaign_performance.to_csv("campaign_performance(new).csv", index=False)
campaign_funnel.to_csv("campaign_funnel(new).csv", index=False)
channel_performance.to_csv("channel_performance(new).csv", index=False)
